# Converting Multi-layer KML to GeoPackage

We have a complex KML file with 2000+ individual layers. Each layer contains only a single feature but having large number of layers means it takes a long time to read and parse them. We can use GeoPandas to read all the layers and merge layers having the same geometry and write the cleaned layers to a GeoPacjage.

## Setup

The following blocks of code will install the required packages and download the datasets to your Colab environment.

In [2]:
try:
    import geopandas
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        !apt install -qq libspatialindex-dev
        !pip install fiona shapely pyproj rtree --quiet
        !pip install geopandas --quiet
    else:
        print('geopandas not found, please install via conda in your environment')

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import fiona

Create a list of layers as a Pandas Series.

In [ ]:
file_path = 'input.kml'
layers = pd.Series(fiona.listlayers(file_path))
layers

'Text [7193A]'

Iterate through each layer and read it using GeoPandas. Create a list of GeoDataFrames for each layer.

This step can take time.

In [ ]:
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'

gdf_list = []

for index, layer in layers.items():
      gdf = gpd.read_file(file_path, layer=layer)
      gdf_list.append(gdf)
      print('Processed item {}: {}'.format(index+1, layer))
      

2182it [08:48,  4.13it/s]


Merge the laers into a single GeoDataFrame.

In [ ]:
merged = pd.concat(gdf_list)
merged

,Name,Description,geometry
0,B2-461,,POINT Z (78.50425 17.68199 0.00000)
0,Point [71939]:0,"<table> <tr> <td align=""right"">Generic Propert...","MULTILINESTRING Z ((78.50424 17.68198 0.00000,..."
0,B2-460,,POINT Z (78.49496 17.68535 0.00000)
0,Point [71936]:0,"<table> <tr> <td align=""right"">Generic Propert...","MULTILINESTRING Z ((78.49496 17.68535 0.00000,..."
0,B2-459,,POINT Z (78.49141 17.69321 0.00000)
...,...,...,...
3,"Marino Foods Pvt Limited, Biscuit Plant, Raja ...",,POINT Z (78.54023 17.63764 0.00000)
4,Real Agro Pvt Ltd Factory,,POINT Z (78.54321 17.63946 0.00000)
5,Sumo Biscuits Pvt Ltd,,POINT Z (78.50614 17.62196 0.00000)
6,Muvao Naturals,,POINT Z (78.52109 17.63342 0.00000)


The geometry column includes different geometries, each geometry type has to be saved into a separate layers. Iterate over goemetry types and write out a layer in the output geopackage.

In [ ]:
output_file = 'merged.gpkg'

In [ ]:
for geomtype in merged.geom_type.unique():
   merged[merged.geom_type == geomtype].to_file(output_file, driver='GPKG', layer=geomtype)